In [2]:
import tensorflow as tf

# tensorflowjs_converter --input_format=tfjs_layers_model --output_format=keras_saved_model ./work/data/tfjs_models/model_4k/model_4k.json ./work/data/keras_models/model_4k/

# load model from tfjs
model = tf.keras.models.load_model('./work/data/keras_models/model_6k/')

2022-05-22 20:31:23.297521: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-22 20:31:23.297540: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-22 20:31:23.297550: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (devitra): /proc/driver/nvidia/version does not exist
2022-05-22 20:31:23.297662: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
import tensorflow_model_optimization as tfmot

# Quantize the model
model = tfmot.quantization.keras.quantize_model(model)

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()


2022-05-22 20:31:25.199575: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmppjbn8a0r/assets


INFO:tensorflow:Assets written to: /tmp/tmppjbn8a0r/assets
2022-05-22 20:31:26.499689: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-05-22 20:31:26.499760: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-05-22 20:31:26.512582: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-05-22 20:31:42.664033: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2022-05-22 20:31:42.664063: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2022-05-22 20:31:44.765767: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:210] disabling MLIR crash reproducer, set env var `MLI

In [6]:
import os

os.makedirs('./work/data/tflite_models/model_6k/', exist_ok=True)
with open('./work/data/tflite_models/model_6k/model_6k.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [11]:
import numpy as np

# load tflite model
interpreter = tf.lite.Interpreter(model_path='./work/data/tflite_models/model_6k/model_6k.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'dense_Dense1_input', 'index': 0, 'shape': array([   1, 6000], dtype=int32), 'shape_signature': array([  -1, 6000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 16, 'shape': array([   1, 6000], dtype=int32), 'shape_signature': array([  -1, 6000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[0.78312075 0.97440666 0.9914669  ... 0.87091523 0.8483699  0.9211155 ]]


In [16]:
import requests

def load_embedding(embedding_name: str, retained_size: int) -> list:
    url = f'http://localhost:3080/embedding?embedding={embedding_name}'
    return requests.get(url).json()['embedding'][:retained_size]

embedding = load_embedding('pymde', 6000)

def scale_rating(rating: int) -> float:
    if rating > 5:
        return 1.0
    else:
        return -1.0

class AnimelistItem:
    def __init__(self, anime_ix: int, rating: int):
        self.anime_ix = anime_ix
        self.rating = rating

    def title(self):
        return embedding[self.anime_ix]['metadata']['title']

    def __str__(self):
        return f'{self.title()}: {self.rating}'

    def __repr__(self):
        return self.__str__()

def load_user_animelist(username: str) -> list:
    url = 'http://localhost:3080/recommendation/training/trainingData'
    payload = [username]
    res = requests.post(url, json=payload).json()['trainingData'][0]
    return [AnimelistItem(rating['animeIx'], rating['rating']) for rating in res if rating.get('rating') is not None and rating.get('animeIx') is not None and rating['animeIx'] < len(embedding)]

def build_model_input(animelist: list) -> list:
    input_arr = [0] * len(embedding)
    for ranking in animelist:
        if ranking.anime_ix < len(embedding):
            input_arr[ranking.anime_ix] = scale_rating(ranking.rating)
    input_tensor = tf.constant([input_arr], dtype=tf.float32)
    return input_tensor

In [17]:
def get_recommendations(username: str, num_recommendations: int) -> list:
    animelist = [] if username is None else load_user_animelist(username)
    input_tensor = build_model_input(animelist)
    prediction = model(input_tensor).numpy()[0].tolist()
    
    # sort by prediction
    sorted_predictions = sorted(enumerate(prediction), key=lambda x: x[1], reverse=True)
    # filter out already watched anime
    filtered_predictions = [x for x in sorted_predictions if x[0] not in [ranking.anime_ix for ranking in animelist]]
    return [AnimelistItem(ix, pred) for ix, pred in filtered_predictions[:num_recommendations]]

In [18]:
get_recommendations('ameo___', 50)

[Death Note: 0.1402461677789688,
 Shingeki no Kyojin: 0.09384071081876755,
 Kimetsu no Yaiba: 0.09384071081876755,
 One Punch Man: 0.09384071081876755,
 Jujutsu Kaisen (TV): 0.09384071081876755,
 Koe no Katachi: 0.09384071081876755,
 Fullmetal Alchemist: Brotherhood: 0.09384071081876755,
 Hunter x Hunter (2011): 0.09384071081876755,
 Toradora!: 0.09384071081876755,
 Code Geass: Hangyaku no Lelouch: 0.09384071081876755,
 Neon Genesis Evangelion: 0.09384071081876755,
 Cowboy Bebop: 0.09384071081876755,
 Fullmetal Alchemist: 0.09384071081876755,
 Shingeki no Kyojin Season 2: 0.04702435061335564,
 Boku no Hero Academia: 0.04702435061335564,
 Shingeki no Kyojin Season 3: 0.04702435061335564,
 Shingeki no Kyojin Season 3 Part 2: 0.04702435061335564,
 Sword Art Online: 0.04702435061335564,
 Boku no Hero Academia 2nd Season: 0.04702435061335564,
 Shingeki no Kyojin: The Final Season: 0.04702435061335564,
 Tokyo Ghoul: 0.04702435061335564,
 Boku no Hero Academia 3rd Season: 0.04702435061335564,